# spconv サンプル

以下のサイトを参考に作成
https://codechina.csdn.net/mirrors/traveller59/spconv/-/blob/master/README.md

In [2]:
import spconv
from torch import nn

In [ ]:
class ExampleNet(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv3d(32, 64, 3), # nn.Conv3dと同じですが、group and all ([d > 1, s > 1])はサポートしていません。
            nn.BatchNorm1d(64), # SparseSequentialでは、非空間的なレイヤーを直接使用することができます。
            nn.ReLU(),
            spconv.SubMConv3d(64, 64, 3, indice_key="subm0"),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            # サブマニフォールドコンボリューションを使用する場合、そのインデックスを共有することでインデックス生成時間を短縮することができます。
            spconv.SubMConv3d(64, 64, 3, indice_key="subm0"),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            spconv.SparseConvTranspose3d(64, 64, 3, 2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            spconv.ToDense() # spconvのテンソルをdenseに変換し、NCHW形式に変換します。
            nn.Conv3d(64, 64, 3),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
        self.shape = shape
    
    def forward(self, features, coors, batch_size):
        coors = coors.int() # torchと違い、このライブラリはint座標しか受け付けません。
        x  = spconv.SparseConvTensor(features, coors, self.shape, batch_size)
        return self.net(x) # dense()